In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer
from tqdm import tqdm
import torch.nn.functional as F

### Utility function

In [2]:
def preprocessing(arguments, key_points, labels):
    columns = ['topic', 'premise', 'hypothesis', 'label']
    data = []
    for index, row in labels.iterrows():
        argument_row = arguments.loc[arguments['arg_id'] == row['arg_id']]
        topic = argument_row['topic'].iloc[0]
        premise = argument_row['argument'].iloc[0]
        kp_row = key_points.loc[key_points['key_point_id'] == row['key_point_id']]
        hypothesis = kp_row['key_point'].iloc[0]
        label = row['label']
        data.append([topic, premise, hypothesis, label])

    result = pd.DataFrame(data, columns=columns)
    return result

### Create Torch Dataset

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, text_df, tokenizer, max_length):
        self.premises = list(text_df['premise'].values+tokenizer.sep_token+text_df['topic'].values)
        self.hypotheses = list(text_df['hypothesis'].values+tokenizer.sep_token+text_df['topic'].values)
        self.labels = list(text_df['label'])
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.premises)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.premises[idx],
            self.hypotheses[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }
        return item


### Define the LightningModule

In [4]:
class KPM(pl.LightningModule):
    def __init__(self, model, learning_rate=2e-5, weight_decay=0.001):
        super(KPM, self).__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
        return outputs

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        outputs = self(input_ids, attention_mask, labels)
        # print("--------------training-------------------------------")
        # print("Shape: "+ str(outputs.logits.shape))
        # print(outputs.logits)
        self.log("train_loss", torch.clone(outputs.loss).detach())
        one_hot_labels = torch.stack([1 - labels, labels], dim=-1)
        one_hot_labels = one_hot_labels.float()
        # loss = outputs.loss
        loss = F.binary_cross_entropy_with_logits(outputs.logits, one_hot_labels, reduction='mean')
        self.log("train_loss", torch.clone(loss).detach())
        return loss

    # def on_validation_batch_start(self, batch, batch_idx, dataloader_idx):
    #     input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
    #     print(len(input_ids))
    #     # inputs = batch[:-1]  # Assuming the input tensor is at the first index
    #     # print(f"Validation Batch {batch_idx + 1}, Input Tensor Size: {inputs[0].size()}")
    #     # print(inputs[0])  # Assuming the input tensor is at index 0

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", torch.clone(outputs.loss).detach())
        one_hot_labels = torch.stack([1 - labels, labels], dim=-1)
        one_hot_labels = one_hot_labels.float()
        # print(labels)
        # loss = outputs.loss
        loss = F.binary_cross_entropy_with_logits(outputs.logits, one_hot_labels, reduction='mean')
        self.log("val_loss", torch.clone(loss).detach())
        return  loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.learning_rate)
        return optimizer

### Import Dataset and Preprocessing

In [5]:
# argument_df = pd.read_csv('./data/arguments_train+dev.csv')
# kp_df = pd.read_csv('./data/key_points_train+dev.csv')
# labels_df = pd.read_csv('./data/labels_train+dev.csv')
#
#
# # val_argument_df = argument_df[argument_df['topic'].isin(dev_topics)]
# # val_kp_df = kp_df[kp_df['topic'].isin(dev_topics)]
# # val_label_df = labels_df[labels_df['topic'].isin(dev_topics)]
# # val_df = preprocessing(val_argument_df, val_kp_df, val_label_df)
# # print(val_argument_df)
# #
# #
# # train_argument_df = argument_df[~argument_df.isin(val_argument_df.to_dict(orient='list')).all(1)]
# # train_kp_df = kp_df[~kp_df.isin(val_kp_df.to_dict(orient='list')).all(1)]
# # train_label_df = labels_df[~labels_df.isin(val_label_df.to_dict(orient='list')).all(1)]
# # train_df = preprocessing(train_argument_df, train_kp_df, train_label_df)
# # print(train_argument_df)
#
# test_argument_df = pd.read_csv('./data/arguments_test.csv')
# test_kp_df = pd.read_csv('./data/key_points_test.csv')
# test_label_df = pd.read_csv('./data/labels_test.csv')
# test_df = preprocessing(test_argument_df, test_kp_df, test_label_df)

### Prediction

In [6]:
#Add new header confidence_score
df = pd.read_csv('./result/KPM_finetuned_model_gpt_group.csv')
print(df.columns)
prediction_df = pd.read_csv('./result/ArgKP21+predictions(v2).csv')
print(prediction_df.columns)

Index(['topic', 'stance', 'argument', 'matched_kp(0_1_2_3)',
       'confidence_score(0_1_2_3)', 'matched_kp(4_7_8_15)',
       'confidence_score(4_7_8_15)', 'matched_kp(5_6_9_10)',
       'confidence_score(5_6_9_10)', 'matched_kp(11_12_13_14)',
       'confidence_score(11_12_13_14)', 'matched_kp(16_17_18_19)',
       'confidence_score(16_17_18_19)', 'matched_kp(20_21_22_23)',
       'confidence_score(20_21_22_23)', 'matched_kp(24_25_26_27)',
       'confidence_score(24_25_26_27)'],
      dtype='object')
Index(['topic', 'stance', 'arguments', 'kep_points', 'predict_kps'], dtype='object')


In [7]:
# df['confidence_score'] = ''
# df.to_csv('./data/KPM_test_data.csv')
# print(f"DataFrame has been saved to {'./data/KPM_test_data.csv'}")

In [12]:
import ast
# Assuming you have already trained the model and saved a checkpoint
checkpoint_path = './KPM/checkpoint/nli_model-epoch=00-val_loss=0.51-24_25_26_27.ckpt'
# Load the model from the checkpoint
model_name = "cross-encoder/nli-distilroberta-base"
num_classes = 2
max_length = 512
batch_size = 16
learning_rate = 5e-05
weight_decay = 0.001

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_classes, ignore_mismatched_sizes = True)
loaded_model = KPM.load_from_checkpoint(model=model, checkpoint_path=checkpoint_path)
print("hello world")

# Set the model to evaluation mode
loaded_model.eval()

df = pd.read_csv('./result/KPM_finetuned_model_gpt_group.csv')
prediction_df = pd.read_csv('./result/ArgKP21+predictions(v2).csv')
for index, row in df.iloc[:1].iterrows():
    print(index)
    kps_df = prediction_df.loc[(prediction_df['topic'] == row['topic']) & (prediction_df['stance'] == row['stance'])]
    kps_list = ast.literal_eval(kps_df['predict_kps'].iloc[0])
    best_kp = ' '
    confidence_score = 0
    for kp in kps_list:
        premise = row['argument']+tokenizer.sep_token+row['topic']
        hypothesis = kp+tokenizer.sep_token+row['topic']
        encoding = tokenizer(
                premise,
                hypothesis,
                truncation=True,
                padding='max_length',
                max_length=max_length,
                return_tensors='pt'
            )
        with torch.no_grad():
            logits = loaded_model(**encoding).logits
            print(F.softmax(logits, dim=1).tolist()[0])
            probabilities = F.softmax(logits, dim=1).tolist()[0][1]
        if probabilities >= confidence_score:
            confidence_score = probabilities
            best_kp = kp
    print(best_kp)
    print(confidence_score)
    df.at[index, 'matched_kp(24_25_26_27)'] = best_kp
    df.at[index, 'confidence_score(24_25_26_27)'] = confidence_score

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cross-encoder/nli-distilroberta-base and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


hello world
7000
[0.9649946689605713, 0.035005345940589905]
[0.9904834032058716, 0.009516608901321888]
[0.9855788946151733, 0.014421097934246063]
[0.003162445966154337, 0.9968374967575073]
[0.9985389709472656, 0.0014609777135774493]
Rather than government interference, individual users and private companies should monitor and regulate content on social media platforms.
0.9968374967575073
7001
[0.9739533066749573, 0.026046747341752052]
[0.6216137409210205, 0.3783862292766571]
[0.9969978332519531, 0.003002149984240532]
[0.13907480239868164, 0.8609251976013184]
[0.9996640682220459, 0.00033588637597858906]
Rather than government interference, individual users and private companies should monitor and regulate content on social media platforms.
0.8609251976013184
7002
[0.0028156570624560118, 0.9971843361854553]
[0.13776028156280518, 0.8622397184371948]
[0.9161936640739441, 0.08380632102489471]
[0.00019301359134260565, 0.9998070597648621]
[0.9990861415863037, 0.000913820113055408]
Rather than

In [13]:
df.to_csv('./result/KPM_finetuned_model_gpt_group.csv', index=False)
print("Add new data successfully!!!!!")

Add new data successfully!!!!!


### Calculate average Score

In [ ]:
df = pd.read_csv('./data/KPM_test_data.csv')
print(df.columns)

In [ ]:
selected_columns = ['4_7_8_15', '0_1_2_3', '5_6_9_10', '11_12_13_14', '16_17_18_19',
       '20_21_22_23', '24_25_26_27']
df['avg'] = df[selected_columns].mean(axis=1)
print(df)

In [ ]:
df.to_csv('./data/KPM_test_data.csv', index=False)
print("Add new data successfully!!!!!")

### Make output json file for automatically evaluating

In [5]:
argument_df = pd.read_csv('./data/arguments_test.csv')
print(len(argument_df))
print(argument_df.columns)

723
Index(['arg_id', 'argument', 'topic', 'stance'], dtype='object')


In [6]:
test_df = pd.read_csv('./data/KPM_test_data.csv')
# test_df['confidence_score'] = 1- test_df['confidence_score']
print(len(test_df))
print(test_df)

3923
                                               topic  stance     arg_id  \
0     Routine child vaccinations should be mandatory      -1    arg_0_0   
1     Routine child vaccinations should be mandatory      -1    arg_0_0   
2     Routine child vaccinations should be mandatory      -1    arg_0_0   
3     Routine child vaccinations should be mandatory      -1    arg_0_0   
4     Routine child vaccinations should be mandatory      -1    arg_0_1   
...                                              ...     ...        ...   
3918            The USA is a good country to live in       1  arg_2_209   
3919            The USA is a good country to live in       1  arg_2_209   
3920            The USA is a good country to live in       1  arg_2_209   
3921            The USA is a good country to live in       1  arg_2_209   
3922            The USA is a good country to live in       1  arg_2_209   

                                               argument key_point_id  \
0     Routine child va

In [20]:
# test_df = test_df.drop(test_df.columns[0], axis=1)
# print(test_df)

                                               topic  stance     arg_id  \
0     Routine child vaccinations should be mandatory      -1    arg_0_0   
1     Routine child vaccinations should be mandatory      -1    arg_0_0   
2     Routine child vaccinations should be mandatory      -1    arg_0_0   
3     Routine child vaccinations should be mandatory      -1    arg_0_0   
4     Routine child vaccinations should be mandatory      -1    arg_0_1   
...                                              ...     ...        ...   
3918            The USA is a good country to live in       1  arg_2_209   
3919            The USA is a good country to live in       1  arg_2_209   
3920            The USA is a good country to live in       1  arg_2_209   
3921            The USA is a good country to live in       1  arg_2_209   
3922            The USA is a good country to live in       1  arg_2_209   

                                               argument key_point_id  \
0     Routine child vaccina

In [23]:
# test_df.to_csv('./data/KPM_test_data.csv', index=False)
# print("Add new data successfully!!!!!")

Add new data successfully!!!!!


In [17]:
output_data = []
arg_id = argument_df['arg_id'].values.tolist()
for arg in arg_id:
    filtered_df = test_df[test_df['arg_id'] == arg]
    kp_id = filtered_df['key_point_id'].values.tolist()
    score = filtered_df['24_25_26_27'].values.tolist()
    result_dict = dict(zip(kp_id, score))
    output_data.append(result_dict)
final_result = dict(zip(arg_id,output_data))
print(final_result)

{'arg_0_0': {'kp_0_0': 0.004264094, 'kp_0_1': 0.000712797, 'kp_0_2': 2.45e-05, 'kp_0_3': 0.020901162}, 'arg_0_1': {'kp_0_0': 0.999077916, 'kp_0_1': 0.000186831, 'kp_0_2': 2.9e-05, 'kp_0_3': 0.038093459}, 'arg_0_2': {'kp_0_0': 0.031888444, 'kp_0_1': 0.00023044, 'kp_0_2': 2.82e-05, 'kp_0_3': 0.936349273}, 'arg_0_3': {'kp_0_0': 0.725610316, 'kp_0_1': 7.07e-05, 'kp_0_2': 3.36e-05, 'kp_0_3': 0.009736656}, 'arg_0_4': {'kp_0_0': 0.8927356, 'kp_0_1': 0.002677533, 'kp_0_2': 0.001524781, 'kp_0_3': 0.056962531}, 'arg_0_5': {'kp_0_0': 0.000332382, 'kp_0_1': 0.001024149, 'kp_0_2': 7.82e-05, 'kp_0_3': 0.000320194}, 'arg_0_6': {'kp_0_0': 0.099606767, 'kp_0_1': 3.39e-05, 'kp_0_2': 2.32e-05, 'kp_0_3': 0.053070199}, 'arg_0_7': {'kp_0_0': 0.999870777, 'kp_0_1': 0.000104631, 'kp_0_2': 2.63e-05, 'kp_0_3': 0.017833358}, 'arg_0_8': {'kp_0_0': 0.412780106, 'kp_0_1': 0.000190485, 'kp_0_2': 2.76e-05, 'kp_0_3': 0.052006453}, 'arg_0_9': {'kp_0_0': 0.257642329, 'kp_0_1': 0.000823184, 'kp_0_2': 0.001105662, 'kp_0_3

In [18]:
import json
file_path = './result/test(24_25_26_27).json'
with open(file_path, 'w') as json_file:
    json.dump(final_result, json_file)
print(f"JSON file '{file_path}' created successfully.")

JSON file './result/test(24_25_26_27).json' created successfully.
